In [27]:
import numpy as np
import pandas as pd
import psycopg2 as pg
from questdb.ingress import Sender
from datetime import datetime
from enum import Enum

In [31]:
class SensorName(Enum):

    ACC = 'accelerometeruncalibrated'
    GYRO = 'gyroscopeuncalibrated'

In [42]:
data = {'messageId': 17, 'sessionId': 'd77381df-416a-4284-986a-be0121fe9453', 'deviceId': '86a5b0e3-6e06-40e2-b226-5a72bd39b65b',
 'payload': [{'name': 'gyroscopeuncalibrated', 'time': 1665756466884661200, 'values': {'z': 0.03507063537836075, 'y': -0.1603156328201294, 'x': -0.08483659476041794}}, {'name': 'accelerometeruncalibrated', 'time': 1665756466884661200, 'values': {'z': -0.7034149169921875, 'y': -0.728485107421875, 'x': -0.061370849609375}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756466904714000, 'values': {'z': 0.08188341557979584, 'y': -0.10671233385801315, 'x': -0.0503835454583168}}, {'name': 'accelerometeruncalibrated', 'time': 1665756466904714000, 'values': {'z': -0.6770782470703125, 'y': -0.7230377197265625, 'x': -0.0795440673828125}}, {'name': 'accelerometeruncalibrated', 'time': 1665756466924767000, 'values': {'z': -0.6923828125, 'y': -0.71722412109375, 'x': -0.0809326171875}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756466924767000, 'values': {'z': 0.05432582274079323, 'y': -0.04422498494386673, 'x': -0.011396997608244419}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756466944820200, 'values': {'z': 0.03211772069334984, 'y': -0.06687303632497787, 'x': 0.02949131280183792}}, {'name': 'accelerometeruncalibrated', 'time': 1665756466944820200, 'values': {'z': -0.69921875, 'y': -0.7032470703125, 'x': -0.085723876953125}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756466964873000, 'values': {'z': 0.02491014264523983, 'y': -0.07187605649232864, 'x': 0.03241785988211632}}, {'name': 'accelerometeruncalibrated', 'time': 1665756466964873000, 'values': {'z': -0.6934356689453125, 'y': -0.693145751953125, 'x': -0.084442138671875}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756466984926200, 'values': {'z': 0.031990423798561096, 'y': -0.09724532812833786, 'x': -0.01787353865802288}}, {'name': 'accelerometeruncalibrated', 'time': 1665756466984926200, 'values': {'z': -0.6973114013671875, 'y': -0.7081146240234375, 'x': -0.0916290283203125}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756467004979200, 'values': {'z': 0.018450113013386726, 'y': -0.07618771493434906, 'x': -0.049133192747831345}}, {'name': 'accelerometeruncalibrated', 'time': 1665756467004979200, 'values': {'z': -0.6914215087890625, 'y': -0.69940185546875, 'x': -0.0895843505859375}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756467025032000, 'values': {'z': -0.007678425870835781, 'y': 0.0024730113800615072, 'x': 0.015121694654226303}}, {'name': 'accelerometeruncalibrated', 'time': 1665756467025032000, 'values': {'z': -0.6683197021484375, 'y': -0.6956634521484375, 'x': -0.102874755859375}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756467045084700, 'values': {'z': -0.01396002434194088, 'y': 0.01601012609899044, 'x': 0.04826926067471504}}, {'name': 'accelerometeruncalibrated', 'time': 1665756467045084700, 'values': {'z': -0.696136474609375, 'y': -0.709228515625, 'x': -0.1010589599609375}}, {'name': 'gyroscopeuncalibrated', 'time': 1665756467065137700, 'values': {'z': -0.036682646721601486, 'y': -0.06903206557035446, 'x': 0.006464823614805937}}, {'name': 'accelerometeruncalibrated', 'time': 1665756467065137700, 'values': {'z': -0.727081298828125, 'y': -0.71246337890625, 'x': -0.07098388671875}}]}

In [43]:
def write_acc(data:dict, db_host:str, db_port:int, table_name:str):

    """
    Write phone accelerometer data to QuestDb 

    Parameters
    ----------
    data : dict
        The raw request data sent by the phone
    db_host : str
        The QuestDb host
    db_port: int
        The QuestDb port
    table_name : str
        The table to write to
    """

    session_id = data['sessionId']
    device_id = data['deviceId']

    for d in data['payload']:

        if d.get("name") == SensorName.ACC.value:

            ts = datetime.fromtimestamp(d["time"] / 1000000000)
            x = d["values"]["x"]
            y = d["values"]["y"]
            z = d["values"]["z"]
            

            with Sender(db_host, db_port) as sender:

                sender.row(
                    table_name,
                    symbols={'device_id':str(device_id),'session_id': str(session_id)},
                    columns={'recorded_timestamp': str(ts) , 'x':x, 'y':y, 'z':z})
                sender.flush()



In [44]:
write_acc(data, "localhost", 9009, "acc")

In [50]:
connection = pg.connect(user="admin",
                            password="quest",
                            host="127.0.0.1",
                            port="8812",
                            database="qdb",
                            options='-c statement_timeout=300000')

In [51]:
data = pd.read_sql("select * from acc", connection)
data.sort_values('recorded_timestamp')

,device_id,session_id,recorded_timestamp,x,y,z,timestamp
0,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:46.884661,-0.061371,-0.728485,-0.703415,2022-10-14 14:35:52.956876
2,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:46.904714,-0.079544,-0.723038,-0.677078,2022-10-14 14:35:52.971747
1,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:46.924767,-0.080933,-0.717224,-0.692383,2022-10-14 14:35:52.971705
7,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:46.944820,-0.085724,-0.703247,-0.699219,2022-10-14 14:35:52.972558
3,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:46.964873,-0.084442,-0.693146,-0.693436,2022-10-14 14:35:52.971775
4,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:46.984926,-0.091629,-0.708115,-0.697311,2022-10-14 14:35:52.972480
5,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:47.004979,-0.089584,-0.699402,-0.691422,2022-10-14 14:35:52.972508
6,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:47.025032,-0.102875,-0.695663,-0.668320,2022-10-14 14:35:52.972529
9,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:47.045085,-0.101059,-0.709229,-0.696136,2022-10-14 14:35:52.972608
8,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,d77381df-416a-4284-986a-be0121fe9453,2022-10-14 09:07:47.065138,-0.070984,-0.712463,-0.727081,2022-10-14 14:35:52.972586


In [52]:
connection.cursor().execute('''drop table acc''')

In [53]:
connection.close()